In [1]:
import os

import pandas as pd
import numpy as np

from dotenv import load_dotenv
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine

In [2]:
dotenv_path = os.path.join(
    os.path.dirname(os.path.abspath('.')),
    '.env'
)

load_dotenv(dotenv_path, verbose=True)

conn_string = os.getenv('DATABASE_URL')

engine = create_engine(conn_string)

In [3]:
%load_ext sql

%sql $conn_string

In [4]:
sql = """
SELECT f.date_id, f.location_id, cases, recoveries, deaths, 
    cases_100k, testing_rate, hospitalization_rate,
    date, year, month, day_of_week, day_of_month,
    country, state, city, latitude, longitude, population
FROM covid_facts f JOIN date_dim d ON d.date_id = f.date_id
JOIN location_dim l ON l.location_id = f.location_id
WHERE country = 'US' AND city IS NULL
ORDER BY state
"""

us_df = pd.read_sql(sql, engine)

In [5]:
us_df.head()

,date_id,location_id,cases,recoveries,deaths,cases_100k,testing_rate,hospitalization_rate,date,year,month,day_of_week,day_of_month,country,state,city,latitude,longitude,population
0,247,84000001,149263,64583.0,2506.0,3022.647524,21989.380372,NaN,2020-09-24,2020,9,3,24,US,Alabama,None,32.3182,-86.9023,4903185.0
1,316,84000001,256828,168387.0,3711.0,5237.983066,32703.701778,NaN,2020-12-02,2020,12,2,2,US,Alabama,None,32.3182,-86.9023,4903185.0
2,284,84000001,193613,81005.0,2967.0,3921.634611,27456.704163,NaN,2020-10-31,2020,10,5,31,US,Alabama,None,32.3182,-86.9023,4903185.0
3,287,84000001,197504,81005.0,2987.0,3995.953651,27799.257014,NaN,2020-11-03,2020,11,1,3,US,Alabama,None,32.3182,-86.9023,4903185.0
4,201,84000001,101466,37923.0,1768.0,2066.697463,15550.116914,11.385122,2020-08-09,2020,8,6,9,US,Alabama,None,32.3182,-86.9023,4903185.0


In [6]:
us_df = us_df.loc[pd.notnull(us_df.population)]
us_df.describe()

,date_id,location_id,cases,recoveries,deaths,cases_100k,testing_rate,hospitalization_rate,year,month,day_of_week,day_of_month,latitude,longitude,population
count,14235.000000,1.423500e+04,1.423500e+04,1.194600e+04,14206.000000,14235.000000,13496.000000,5129.000000,14235.0,14235.000000,14235.000000,14235.000000,14235.000000,14235.000000,1.423500e+04
mean,194.334387,7.628753e+07,9.179591e+04,3.901173e+04,2741.854709,1410.292177,22237.818284,12.143891,2020.0,7.557640,2.991992,15.853109,36.800258,-84.992940,6.024244e+06
std,75.292355,2.425706e+07,1.600239e+05,8.615791e+04,5196.191790,1453.418745,22475.224955,5.245396,0.0,2.488042,2.001073,8.688430,10.796215,49.687909,8.994377e+06
min,1.000000,1.600000e+01,0.000000e+00,0.000000e+00,0.000000,0.000000,5.391708,1.418440,2020.0,1.000000,0.000000,1.000000,-14.271000,-170.132000,5.514400e+04
25%,132.000000,8.400001e+07,5.437000e+03,1.573000e+03,121.000000,266.184421,5761.788808,8.359942,2020.0,6.000000,1.000000,8.000000,33.856900,-105.311100,1.344212e+06
50%,195.000000,8.400003e+07,3.099500e+04,8.365500e+03,734.000000,990.979411,15907.304838,11.282093,2020.0,8.000000,3.000000,16.000000,39.059800,-86.902300,3.565287e+06
75%,259.000000,8.400004e+07,1.105510e+05,4.152750e+04,2978.500000,2097.794404,30031.790665,15.268243,2020.0,10.000000,5.000000,23.000000,42.230200,-76.802100,7.278717e+06
max,322.000000,8.400006e+07,1.415396e+06,1.050416e+06,35118.000000,11710.000000,157008.611795,38.501190,2020.0,12.000000,6.000000,31.000000,61.370700,145.673900,3.294663e+08


In [7]:
states_df = pd.read_csv('https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv')
states_df.head()

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


In [8]:
states_df = states_df.rename(columns=lambda col: col.lower())
states_df.head()

,state,state code,region,division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


In [9]:
states_df.region.unique()

array(['West', 'South', 'Northeast', 'Midwest'], dtype=object)

In [10]:
us2_df = us_df.join(states_df.set_index('state'), on='state').sort_values(['state', 'date'])
us2_df.head()

,date_id,location_id,cases,recoveries,deaths,cases_100k,testing_rate,hospitalization_rate,date,year,...,day_of_month,country,state,city,latitude,longitude,population,state code,region,division
103,52,84000001,5,0.0,0.0,5.0,NaN,NaN,2020-03-13,2020,...,13,US,Alabama,None,32.3182,-86.9023,4903185.0,AL,South,East South Central
165,53,84000001,6,0.0,0.0,6.0,NaN,NaN,2020-03-14,2020,...,14,US,Alabama,None,32.3182,-86.9023,4903185.0,AL,South,East South Central
90,54,84000001,12,0.0,0.0,12.0,NaN,NaN,2020-03-15,2020,...,15,US,Alabama,None,32.3182,-86.9023,4903185.0,AL,South,East South Central
134,55,84000001,29,0.0,0.0,29.0,NaN,NaN,2020-03-16,2020,...,16,US,Alabama,None,32.3182,-86.9023,4903185.0,AL,South,East South Central
14,56,84000001,39,0.0,0.0,39.0,NaN,NaN,2020-03-17,2020,...,17,US,Alabama,None,32.3182,-86.9023,4903185.0,AL,South,East South Central


In [11]:
us2_df['cases_norm100k'] = us2_df.cases / (us2_df.population / 100_000)
us2_df.head()

,date_id,location_id,cases,recoveries,deaths,cases_100k,testing_rate,hospitalization_rate,date,year,...,country,state,city,latitude,longitude,population,state code,region,division,cases_norm100k
103,52,84000001,5,0.0,0.0,5.0,NaN,NaN,2020-03-13,2020,...,US,Alabama,None,32.3182,-86.9023,4903185.0,AL,South,East South Central,0.101975
165,53,84000001,6,0.0,0.0,6.0,NaN,NaN,2020-03-14,2020,...,US,Alabama,None,32.3182,-86.9023,4903185.0,AL,South,East South Central,0.122369
90,54,84000001,12,0.0,0.0,12.0,NaN,NaN,2020-03-15,2020,...,US,Alabama,None,32.3182,-86.9023,4903185.0,AL,South,East South Central,0.244739
134,55,84000001,29,0.0,0.0,29.0,NaN,NaN,2020-03-16,2020,...,US,Alabama,None,32.3182,-86.9023,4903185.0,AL,South,East South Central,0.591452
14,56,84000001,39,0.0,0.0,39.0,NaN,NaN,2020-03-17,2020,...,US,Alabama,None,32.3182,-86.9023,4903185.0,AL,South,East South Central,0.795401


In [ ]:
us2_df['recoveries_norm100k'] = us2_df.recoveries / round((us2_df.population / 100_000))
us2_df.head

In [15]:
us2_df.head()

,date_id,location_id,cases,recoveries,deaths,cases_100k,testing_rate,hospitalization_rate,date,year,...,city,latitude,longitude,population,Region,Midwest,Northeast,South,West,cases_norm100k
Alabama,234,84000001,136107,54223.0,2333.0,2788.044914,20617.659746,NaN,2020-09-11,2020,...,None,32.3182,-86.9023,4903185.0,South,0,0,1,0,2775.889549
Alabama,234,84000001,136107,54223.0,2333.0,2788.044914,20617.659746,NaN,2020-09-11,2020,...,None,32.3182,-86.9023,4903185.0,South,0,0,1,0,2775.889549
Alabama,234,84000001,136107,54223.0,2333.0,2788.044914,20617.659746,NaN,2020-09-11,2020,...,None,32.3182,-86.9023,4903185.0,South,0,0,1,0,2775.889549
Alabama,234,84000001,136107,54223.0,2333.0,2788.044914,20617.659746,NaN,2020-09-11,2020,...,None,32.3182,-86.9023,4903185.0,South,0,0,1,0,2775.889549
Alabama,234,84000001,136107,54223.0,2333.0,2788.044914,20617.659746,NaN,2020-09-11,2020,...,None,32.3182,-86.9023,4903185.0,South,0,0,1,0,2775.889549


In [16]:
us_df.head()

,date_id,location_id,cases,recoveries,deaths,cases_100k,testing_rate,hospitalization_rate,date,year,month,day_of_week,day_of_month,country,state,city,latitude,longitude,population
0,234,84000001,136107,54223.0,2333.0,2788.044914,20617.659746,NaN,2020-09-11,2020,9,4,11,US,Alabama,None,32.3182,-86.9023,4903185.0
1,218,84000001,119904,48028.0,2045.0,2432.174189,19246.367412,11.743841,2020-08-26,2020,8,2,26,US,Alabama,None,32.3182,-86.9023,4903185.0
2,296,84000001,210637,88038.0,3213.0,4295.921936,28993.195239,NaN,2020-11-12,2020,11,3,12,US,Alabama,None,32.3182,-86.9023,4903185.0
3,138,84000001,20777,11395.0,692.0,418.095585,5293.824320,9.863415,2020-06-07,2020,6,6,7,US,Alabama,None,32.3182,-86.9023,4903185.0
4,274,84000001,176605,74439.0,2828.0,3582.854818,25973.382607,NaN,2020-10-21,2020,10,2,21,US,Alabama,None,32.3182,-86.9023,4903185.0
